In [336]:
 import math 
from sympy import*
from sympy.abc import*

def F_z(y):
    H = 0.012
    z = y / (math.sqrt(2) * H)
    F_z = (2 / math.sqrt(math.pi)) * (integrate(exp(-(x ** 2)), (x, 0, z)).evalf())
    return F_z

def dolya(x1, x2, y1, y2):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (F_z(y2) - F_z(y1))
    return ras

def dolya_prist(x1, x2, y1):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (1 - F_z(y1))
    return ras
def okrug(x):
    x = x * 10000
    ost = x % 1
    if ost >= 0.5:
        x = (x // 1) + 1
    else: 
        x = x // 1
    if x == 0:
        x = 1
    x /= 10000
    return x

def nax_fi(alfa):
    fi = 0.1
    t = math.tan(alfa * math.pi / 180)
    A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
    mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
    t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)
                                 ) ** 2) - (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
    ta = (t * 1000) // 1
    ta1 = (t1 * 1000) // 1
    while ta != ta1:
        t = math.tan(alfa * math.pi / 180)
        A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
        mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
        t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)) ** 2) -
                                    (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
        fi += 0.00001
        ta = (t * 1000) // 1
        ta1 = (t1 * 1000) // 1
    return fi

## Исходные данные

$P_п = 100 кН;~ p_к = 15 МПа;~p_a = 0.1 МПа \\Компоненты~топлива ~~~~ АТ + НДМГ$

In [337]:
P_pust = 100000
p_k = 15 ** 6
p_a = 0.1 * 10**6

## Определение основных парраметров

### в ядре потока

$\alpha_{опт.я} = 0.94;~ K_0 = 3.062;~ K_m = 2.878;~ I_уп = 3190.8~\frac{м}{c}; \\ T_я = 3517.68K; R= 338.4 \frac{Дж}{кг \cdot К};~ k= 1.21$

In [338]:
alfa_optim_yadr = 0.94
K_0 = 3.0620
K_m = alfa_optim_yadr * K_0
I_u_yadr = 3190.8
T_yadr = 3517.68
R = 338.431
k = 1.21

### В пристенеке 

$ T_{прист} = 2000 К;~ \alpha_{прист} = 0.34;~ K_{mприст} = 1.04108;~ I_{уприст} = 2721.04$

In [339]:
T_prist = 2000
alfa_prist = 0.34
K_m_prist = alfa_prist * K_0
I_u_prist = 2721.04

### Параметры с учетом избытка горючего в пристенке
$ I_{уд} = 3167.312~ \frac{м}{с};~~ \dot m_{\Sigma} = 31.57~ \frac{кг}{с}$

In [340]:
I_ud = I_u_yadr - (0.05 * (I_u_yadr - I_u_prist))
m_sum = P_pust / I_ud

#### Расходы в пристенке 

$ \dot m_{прист} = 1.578~\frac{кг}{с};~ \dot m_{гприст} = 0.7734~\frac{кг}{с}~ \dot m_{окприст} = 0.8052~\frac{кг}{с}$

In [341]:
m_prist = m_sum * 0.05
m_g_prist = m_prist / (K_m_prist + 1)
m_ok_prist = m_prist - m_g_prist

#### Расходы в ядре 

$ \dot m_{я} = 29.999 \frac{кг}{с};~ \dot m_{гя} = 7.73 \frac{кг}{с};~ \dot m_{окя} = 22.26 \frac{кг}{с}$

In [342]:
m_yadr = m_sum - m_prist
m_g_yadr = m_yadr / (K_m + 1)
m_ok_yadr = m_yadr - m_g_yadr

#### Расходный комплекс

$ A_k = 0.6504;~~ \beta = 1677.4\frac{м}{с}$

In [343]:
A_k = (k ** 0.5) * ((2 / (k + 1)) ** ((k + 1)  / (2 * (k - 1))))
betta = ((T_yadr * R) ** 0.5) / A_k

#### Параметры критического сечения 

$F_{кр} = 0.0046 м^2;~ D_{кр} = 0.0769 м$

In [344]:
F_kr = (m_sum * betta) / p_k
D_kr = ((4 * F_kr) / 3.14) ** 0.5

#### Параметры форсуночной головки
##### Находим радиус форсуночной головкии и задаемся шагом между форсунками

$F_г = 0.0186 м^2;~ D_г = 0.154 м;~ R_г = 0.077м;~ H = 0.012м;~ D_ш = 0.124м;$

In [345]:
F_g = 4 * F_kr
D_g = ((4 * F_g) / 3.14) ** 0.5
R_g = D_g / 2
H = 0.012
D_sh = D_g - (2.5 * H)

##### Определяем радиус расположения пристеночных форсунок 

$R_{прист} = 0.0710 м;$

In [346]:
R_prist = (D_g - H) / 2

#### Задаемся количеством форсунок в ядре и пристенке 
$n_{гпр} = 108;~ n_{окпр} = 36;~ n_{гя} = 65;~ n_{окпр} = 138;~$

In [347]:
n_g_prist = 108
n_ok_prist = 36
n_g_yadr = 85
n_ok_yadr =168 

#### Определеим соответствующие расходы через форсунки 

$\dot m_{фгя} = 0.0909 \frac{кг}{с};~ \dot m_{фокя} = 0.1325 \frac{кг}{с};~\dot m_{фгприст} = 0.0072 \frac{кг}{с};~\dot m_{я} = 0.0048 \frac{кг}{с};~$

In [348]:
m_fg_yadr = m_g_yadr / n_g_yadr
m_fok_yadr = m_ok_yadr / n_ok_yadr
m_fg_prist = m_g_prist / n_g_prist
m_fok_prist = m_ok_prist / n_ok_yadr

### Геометрический метод расчета соотношения компонентов в ядре потока

#### Для площадки с форсункой горючего по центру и форсунками окислителя по краям квадрата справедливо

$K_{km~геом} = 2.913$

In [349]:
n_o = 6 # Количество форсунок окислителя
m_otn_ok = 1 / 3 # Относительный расход каждой форсунки окислителя на площадку
n_g = 1 # Количество форсунок окислителя 
m_otn_g = 1 # Относительный расход каждой форсунки горючего на площадку
k_m_yadr_geom = (n_o * m_otn_ok * m_fok_yadr) / (n_g * m_otn_g * m_fg_yadr)

### Расчет соотношения компонентов методом Иевлева

#### Приняв что ддля каждого элемента принимабт участие форсунки, которые находятся на растояни 3-х шагов 

##### Площадка №1 (центральная)
$K_{m1} = 2.524$

In [350]:
XX1ok = [(-0.006, 0.006), (-0.006, 0.006), (-0.0121, -0.0000928),
       (-0.0178, -0.005814), (-0.017814, -0.005814), (-0.01209, -0.000928),
        (-0.0121, -0.0000928),
       (-0.0178, -0.005814), (-0.017814, -0.005814), (-0.01209, -0.000928),(-0.0121, -0.0000928),
       (-0.0178, -0.005814), (-0.017814, -0.005814), (-0.01209, -0.000928),(-0.0121, -0.0000928),
       (-0.0178, -0.005814), (-0.017814, -0.005814), (-0.01209, -0.000928),(-0.0121, -0.0000928),
       (-0.0178, -0.005814), (-0.017814, -0.005814), (-0.01209, -0.000928), 
        (-0.017814, -0.005814), (-0.01209, -0.000928)]
YY1ok = [(-0.0198, -0.0078), (-0.01298, 0.00098), (-0.023375, -0.011375), 
       (-0.0198, -0.00780), (-0.01298, -0.00098), (-0.00941, 0.002589), 
        (-0.023375, -0.011375), 
       (-0.0198, -0.00780), (-0.01298, -0.00098), (-0.00941, 0.002589),
        (-0.023375, -0.011375), 
       (-0.0198, -0.00780), (-0.01298, -0.00098), (-0.00941, 0.002589),
        (-0.023375, -0.011375), 
       (-0.0198, -0.00780), (-0.01298, -0.00098), (-0.00941, 0.002589),
        (-0.023375, -0.011375), 
       (-0.0198, -0.00780), (-0.01298, -0.00098), (-0.00941, 0.002589), 
        (-0.01298, -0.00098), (-0.00941, 0.002589)]
d_sum = 0
for i1 in range(len(XX1ok)):
    d = dolya(XX1ok[i1][0], XX1ok[i1][1], YY1ok[i1][0], YY1ok[i1][1])
    d_sum += d
m_ok1 = m_fok_yadr * d_sum

XX1g = [(-0.006, 0.006), (-0.006, 0.006)]
YY1g = [(-0.006, 0.006), (-0.02679, -0.01479)]
d_sum = 0
for i1 in range(len(XX1g)):
    d = dolya(XX1g[i1][0], XX1g[i1][1], YY1g[i1][0], YY1g[i1][1])
    d_sum += 6*d
m_g1 = m_fg_yadr * d_sum
k_m1 = m_ok1 / m_g1

#### Площадки №2, 3, 4 по радиуссу:
#### Распределение аналогично распределению площади №1, ввиду той же геометрии расположения форсунок в рассматриваемой зоне 3Н

#### Площадка №5 по радиусу:
$K_{m5} = 1.51$

In [351]:
XX5ok_yadr = [(-0.006, 0.006), (-0.006, 0.006), (0.0000923, 0.012093),
       (0.006, 0.018), (0.006, 0.018), (0.0000923, 0.01292), 
             (-0.006, 0.006), (-0.006, 0.006), (0.0000923, 0.012093),
       (0.006, 0.018), (0.006, 0.018), (0.0000923, 0.01292), 
             (0.0000923, 0.012093), (0.0000923, 0.012093),(0.0000923, 0.012093),
             (0.0000923, 0.012093),]
YY5ok_yadr = [(-0.01298, -0.000982), (-0.0198, -0.0078), (-0.0234, -0.0114), 
       (-0.019803, -0.00783), (-0.0129, -0.0098), (-0.0094, 0.002589), 
             (-0.01298, -0.000982), (-0.0198, -0.0078), (-0.0234, -0.0114), 
       (-0.019803, -0.00783), (-0.0129, -0.0098), (-0.0094, 0.002589), 
              (-0.0234, -0.0114),  (-0.0234, -0.0114),  (-0.0234, -0.0114), 
              (-0.0234, -0.0114)]
d_sum_yadr = 0

for i5 in range(len(XX5ok_yadr)):
    d = dolya(XX5ok_yadr[i5][0], XX5ok_yadr[i5][1], YY5ok_yadr[i5][0], YY5ok_yadr[i5][1])
    d_sum_yadr += d
m_ok5_yadr = m_fok_yadr * d_sum_yadr

In [352]:
XX5g_yadr = [(-0.0065, 0.0065), (-0.0144, -0.00043)]
YY5g_yadr = [(-0.0065, 0.0065), (-0.0185, -0.0055)]
d_sum_yadr = 0

for i5 in range(len(XX5g_yadr)):
    d = dolya(XX5g_yadr[i5][0], XX5g_yadr[i5][1], YY5g_yadr[i5][0], YY5g_yadr[i5][1])
    d_sum_yadr += 4*d
m_g5_yadr = m_fg_yadr * d_sum_yadr
k_m5 = m_ok5_yadr / m_g5_yadr

#### Площадка № 6 (нижняя по пристеночному слою)
$K_{m6} = 1.08$

In [353]:
XX6ok_yadr = [(-0.002589, 0.00941)]
YY6ok_yadr = [0.012]
d_sum_yadr = 0

for i6 in range(len(XX6ok_yadr)):
    d = dolya_prist(XX6ok_yadr[i6][0], XX6ok_yadr[i6][1], YY6ok_yadr[i6])
    d_sum_yadr += 2*d

In [354]:
XX6ok_prist = [(0.00564, 0.017634), (-0.006, 0.006), (-0.01764, -0.005654)]
YY6ok_prist = [0.00002, -0.001, 0.00002]
d_sum_prist = 0

for i6 in range(len(XX6ok_prist)):
    d = dolya_prist(XX6ok_prist[i6][0], XX6ok_prist[i6][1], YY6ok_prist[i6])
    d_sum_prist += d
m_6ok = (d_sum_yadr * m_fok_yadr) + (d_sum_prist * m_fok_prist)

In [355]:
XX6g_yadr = [(0.011344, 0.023344)]
YY6g_yadr = [0.012]
d_sum_yadr = 0

for i6 in range(len(XX6g_yadr)):
    d = dolya_prist(XX6g_yadr[i6][0], XX6g_yadr[i6][1], YY6g_yadr[i6])
    d_sum_yadr += 2*d


In [356]:
XX6g_prist = [(-0.0246, -0.012635), (0.012635, 0.024635), (0.006503, 0.018503),
             (-0.006503, 0.018503), (-0.00016, 0.01184), (-0.01184, 0.00016),
             (-0.012275, -0.000275), (0.000275, 0.012275), (-0.006, 0.006)]
YY6g_prist = [-0.00354, -0.00354, -0.004906,
              -0.004906, -0.000745, -0.000745, 
              -0.005726, 0.005726, -0.006]
d_sum_prist = 0

for i6 in range(len(XX6g_prist)):
    d = dolya_prist(XX6g_prist[i6][0], XX6g_prist[i6][1], YY6g_prist[i6])
    d_sum_prist += d
m_6g = (d_sum_yadr * m_fg_yadr) + (d_sum_prist * m_fg_prist)
km_6 = m_6ok / m_6g

#### Площадка № 7 (средняя по пристеночному слою)
$K_{m7} = 0.8$

In [357]:
XX7ok_yadr = [(-0.008831, 0.003169)]
YY7ok_yadr = [0.009432]
d_sum_yadr = 0

for i7 in range(len(XX7ok_yadr)):
    d = dolya_prist(XX7ok_yadr[i7][0], XX7ok_yadr[i7][1], YY7ok_yadr[i7])
    d_sum_yadr += d
m_7ok_yadr = d_sum_yadr * m_fok_yadr

In [358]:
XX7ok_prist = [(-0.02334, -0.01134), (0.01134, 0.02334),
             (-0.00016, 0.01184), (-0.01104, 0.00016)]
YY7ok_prist = [0.001282, 0.001282, -0.000745, -0.000745]
d_sum_prist = 0

for i7 in range(len(XX7ok_prist)):
    d = dolya_prist(XX7ok_prist[i7][0], XX7ok_prist[i7][1], YY7ok_prist[i7])
    d_sum_prist += d
m_7ok_prist = d_sum_prist * m_fok_prist
m_7ok = m_7ok_prist + m_7ok_yadr

In [359]:
XX7g_yadr = [(-0.002086, 0.009914)]
YY7g_yadr = [0.01124]
d_sum_yadr = 0

for i7 in range(len(XX7g_yadr)):
    d = dolya_prist(XX7g_yadr[i7][0], XX7g_yadr[i7][1], YY7g_yadr[i7])
    d_sum_yadr += d
m_7g_yadr = d_sum_yadr * m_fg_yadr

In [360]:
XX7g_prist = [(-0.012635, -0.024635), (0.012635, 0.024635), 
              (-0.017634, -0.005634), (0.005634, 0.017634), (-0.018503, -0.006503),
              (0.006503, 0.018503), (-0.012275, -0.000275), (0.000275, 0.012275),
              (-0.006, 0.006), (-0.006, 0.006)]
YY7g_prist = [-0.003547, -0.003547, 0.000017, 0.000017, -0.004906,
              -0.004906, -0.005726, -0.005726, -0.001, -0.006]
d_sum_prist = 0

for i7 in range(len(XX7g_prist)):
    d = dolya_prist(XX7g_prist[i7][0], XX7g_prist[i7][1], YY7g_prist[i7])
    d_sum_prist += d
m_7g_prist = d_sum_prist * m_fg_prist
m_7g = m_7g_prist + m_7g_yadr
km_7 = m_7ok / m_7g

#### Площадка № 8 (верхняя по пристеночному слою)
$K_{m8} = 1.06$

In [361]:
XX8ok_yadr = [(-0.018094, -0.006094), (0.005859, 0.006141), (0.005906, 0.017906)]
YY8ok_yadr = [0.010627, 0.010707, 0.010627]
d_sum_yadr = 0

for i8 in range(len(XX8ok_yadr)):
    d = dolya_prist(XX8ok_yadr[i8][0], XX8ok_yadr[i8][1], YY8ok_yadr[i8])
    d_sum_yadr += d
m_8ok_yadr = d_sum_yadr * m_fok_yadr

In [362]:
XX8ok_prist = [(0.005634, 0.017634), (-0.017634, -0.005634), (-0.006, 0.006)]
YY8ok_prist = [0.000017, 0.000017, 0.001]
d_sum_prist = 0

for i8 in range(len(XX8ok_prist)):
    d = dolya_prist(XX8ok_prist[i8][0], XX8ok_prist[i8][1], YY8ok_prist[i8])
    d_sum_prist += d

m_8ok_prist = d_sum_prist * m_fok_prist
m_8ok = m_8ok_prist + m_8ok_yadr

In [363]:
XX8g_yadr = [(-0.0120479, -0.0000479), (0.0000479, 0.0120479)]
YY8g_yadr = [0.0146, 0.0146]
d_sum_yadr = 0

for i8 in range(len(XX8g_yadr)):
    d = dolya_prist(XX8g_yadr[i8][0], XX8g_yadr[i8][1], YY8g_yadr[i8])
    d_sum_yadr += d
m_8g_yadr = d_sum_yadr * m_fg_yadr

In [364]:
XX8g_prist = [(-0.01134, -0.02334), (0.01134, 0.02334),(-0.024635, -0.012635),(0.012635, 0.024635),
             (-0.018502, -0.006502), (0.006502, 0.018502), (-0.01184, 0.00016), (-0.00016, 0.01184),
             (-0.012275, 0.000275), (0.000275, 0.012275), (-0.006, 0.006)]
YY8g_prist = [0.001283, 0.001283, 0.003547, 0.003547, 0.004906, 0.004906, 0.000745, 0.000745, 
             0.005726, 0.005726, 0.006]
d_sum_prist = 0

for i8 in range(len(XX8g_prist)):
    d = dolya_prist(XX8g_prist[i8][0], XX8g_prist[i8][1], YY8g_prist[i8])
    d_sum_prist += d
m_8g_prist = d_sum_prist * m_fg_prist
m_8g = m_8g_prist + m_8g_yadr
km_8 = m_8ok / m_8g

### Расчет центробежной форсунки окислителя в ядре

$\Delta p_ф = 2~МПа;~ \rho_{ок} = 1450 \frac{кг}{м^3};~\alpha = 45^{\circ}$



In [365]:
delta_p_f = 2 * 10 ** 6
ro_ok = 1140 
din_vyaz = 4.2 * (10**-4)
alfa = 45

### Диаметр форсунки и толщина боковой стенки 
$D_ф = 6мм;~~ \delta_{ст}=1.5мм$

In [366]:
D_f = H / 2
delt = 0.0015

### Димаетр камеры закручивания, входного тангенциального отверстия и число тангенциальных отверстий
$D_{кз}=4мм;~~d_{вх}=1мм;~~i_{вх}=4$

In [367]:
D_kz = D_f - (2 * delt)
d_vx = 0.001
r_vx = d_vx / 2
i_vx = 4

### Длина тангенциального отверстия 

$l_{вх}= 1.71мм$

In [368]:
l_vx = ((((D_f/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)-((((D_kz/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)

### Длина камеры закручивания, принимаем ${l'}_{кз} = 1.4$ 
$\\ l_{кз}=4.2мм$


In [394]:
l_sht_kz = 1.4
l_kz = l_sht_kz * D_kz

### угол наклона образующей конической поверхности перехода от камеры закручивания к соплу 
$\phi_c=45^\circ$

In [395]:
fi_c = 45

### Величина радиуса, на котором расположена ось входного тангеницального отверстия от оси форсунке
$R_{вх} = 1мм$

In [396]:
R_vx = (D_kz-d_vx) / 2

### Диаметр сопла форсунки 
$d_c = 2.1мм$

In [397]:
d_c = 0.7*D_kz

### Относительная длина сопла ${l'}_c=0.6$
### Длина сопла
$l_c=1.26мм$

In [385]:
l_shc=0.6
l_c = l_shc*d_c

### Угол между осью входного тангенциального отверстия и наппавлением оси форсунки $\theta = 90^{\circ}$

### Геометрическая характеристика центробежной форсунки
$A=1.05$

In [386]:
theta = 90
A = ((R_vx*(d_c/2)) / (i_vx*((d_vx/2)**2))) * math.sin(3.14*theta/180)

### Число Рейнольдса на входе в форсунку 
$Re_{вх} = 2.008 \cdot 10^5$

In [387]:
Re = (4 * m_fok_yadr) / (math.pi * din_vyaz * d_vx * math.sqrt(i_vx))

### Коэффициент трения при условиях на входе в форсунку
$\lambda = 0.192$

In [388]:
lambd = math.exp(((25.8) / ((math.log10(Re)) ** 2.58)) - 2)

### Эквивалентная геометрическая характеристика форсунки, учитывающая влияние вязкости реальной жидкости
$A_э = 1.16$

In [389]:
A_e = A / (1 + ((lambd/2) * ((((R_vx / (r_vx * i_vx)) ** 2) * (1 / i_vx)) - A)))

### Коэффициент живого сечения сопла форсунки 
$\varphi = 0.61$

In [390]:
fi = 1/ (((((A_e / math.sqrt(8)) + math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3)) + \
     (((A_e / math.sqrt(8)) - math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3))) ** 2)

### Коэффициент расхода в центробежной форсунки
$\mu = 0.404$

In [391]:
mu_fi = fi * ((fi / (2 - fi)) ** 0.5)

### Длина форсунки 
$h = 5.9мм;$

In [400]:
h = l_kz + l_c + (((D_kz - d_c) / 2) * math.tan(math.pi * fi_c / 180))